In [1]:
# The script aims to descibe the properties available in a suburb and find rental yields
import pandas as pd
import numpy as np


In [ ]:
# pip install -e git+https://github.com/tomquirk/realestate-com-au-api.git#egg=realestate_com_au_api

from realestate_com_au import RealestateComAu

api = RealestateComAu()

# Get property listings
listings = api.search(locations=["seventeen seventy, qld 4677"], channel="buy", keywords=["tenant"], exclude_keywords=["pool"])

In [ ]:
df = api.search(channel="buy")


In [ ]:
# Part 1: download information
# df_rent = api.search(locations=['2144,AUBURN'],channel="rent")
# df_all = api.search(locations=['2144,AUBURN'])
# last dowload was 6.24 minutes for both lines

# df_rent = pd.DataFrame(df_rent)
# df_rent1 = df_rent.reset_index()
# df_rent2 = df_rent1.drop(columns=['id','index','images','images_floorplans','listers'])
# df_rent_Unique = df_rent2.drop_duplicates()

# df_all = pd.DataFrame(df_all)
# df_all1 = df_all.reset_index()
# df_all2 = df_all1.drop(columns=['id','index','images','images_floorplans','listers'])
# df_all_Unique = df_all2.drop_duplicates()

# del df_rent1, df_rent2, df_all1, df_all2

In [36]:
# part 2.1 create masterlist of postcode and suburbs

REF_PCODE_SBRB = df_all_Unique.append([df_all_Unique],ignore_index=True).reset_index() 
# REF_PCODE_SBRB = REF_PCODE_SBRB[["postcode","suburb"]]
# full comparison version
REF_PCODE_SBRB = REF_PCODE_SBRB[["postcode","suburb","property_type","bedrooms"]]
REF_PCODE_SBRB = REF_PCODE_SBRB.drop_duplicates()


In [37]:
# Part 2.2 Remove rentals from the masterlist

A_FIND_BUY = df_all_Unique[df_all_Unique['price']>10000]
INFO_BUY = A_FIND_BUY.reset_index()
# INFO_BUY.columns = INFO_BUY.columns.str.replace("_x","")

In [38]:
# Part 2.3: format rental and sales data to calculate yield

# Rental data
INFO_RENT_1 = df_rent_Unique[['suburb','state','postcode','short_address','full_address','property_type','price','price_text','bedrooms','bathrooms','parking_spaces','building_size','building_size_unit','land_size','land_size_unit']].fillna(0)

# INFO_RENT_1 = R_INFO_ALL.drop(columns=['badge','url','listing_company_name','listing_company_phone','description']).fillna(0)
INFO_RENT_1 = INFO_RENT_1.drop_duplicates()
# summarise by postcode and suburb

INFO_RENT_MED = INFO_RENT_1.groupby(["postcode","suburb","property_type","bedrooms"])["price"].median().reset_index()
INFO_RENT_MED.rename(columns={'price':'price_RENT'}, inplace = True)

INFO_RENT_FREQ = INFO_RENT_1.groupby(["postcode","suburb","property_type","bedrooms"])["full_address"].count().reset_index()
INFO_RENT_FREQ.rename(columns={'full_address':'num_RENT'}, inplace = True)


# Sales data
INFO_BUY_1 = INFO_BUY[['suburb','state','postcode','short_address','full_address','property_type','price','price_text','bedrooms','bathrooms','parking_spaces','building_size','building_size_unit','land_size','land_size_unit']].fillna(0)
# A_INFO_SUBSET= INFO_BUY_.reset_index()
INFO_BUY_1.drop_duplicates()
# summarise by postcode and suburb

INFO_BUY_MED = INFO_BUY_1.groupby(["postcode","suburb","property_type","bedrooms"])["price"].median().reset_index()
INFO_BUY_MED.rename(columns={'price':'price_BUY'}, inplace = True)

INFO_BUY_FREQ = INFO_BUY_1.groupby(["postcode","suburb","property_type","bedrooms"])["full_address"].count().reset_index()
INFO_BUY_FREQ.rename(columns={'full_address':'num_BUY'}, inplace = True)

# del INFO_RENT_1, INFO_BUY_1



In [39]:
# part 2.4: calculate rental yield

df_comp = REF_PCODE_SBRB.merge(INFO_BUY_MED,how='left',on=["postcode","suburb","property_type","bedrooms"]).merge(INFO_RENT_MED,how='left',on=["postcode","suburb","property_type","bedrooms"]).fillna(0)
df_comp = df_comp.merge(INFO_BUY_FREQ,how='left',on=["postcode","suburb","property_type","bedrooms"]).merge(INFO_RENT_FREQ,how='left',on=["postcode","suburb","property_type","bedrooms"]).fillna(0)

df_comp.dtypes
# df_comp = REF_PCODE_SBRB.merge(R_INFO_SUBSET_MED,how='left',on=["postcode","suburb"]).merge(R_INFO_SUBSET_FREQ,how='left',on=["postcode","suburb"]).fillna(0)
# df_comp = df_comp.merge(A_INFO_SUBSET_MED,how='left',on=["postcode","suburb"]).merge(A_INFO_SUBSET_FREQ,how='left',on=["postcode","suburb"]).fillna(0)


df_comp = df_comp.loc[(df_comp['num_RENT']>1) | (df_comp['num_BUY']>1)]
df_comp['num_RENT'] = df_comp['num_RENT'].astype('int')
df_comp.dtypes

df_comp['TOTAL_AVAILABLE'] = df_comp['num_RENT'] + df_comp['num_BUY']

df_comp['YEARLY_RENT']=df_comp['price_RENT']*52
df_comp['RENT_YIELD']=df_comp['YEARLY_RENT']/df_comp['price_BUY']

df_comp.replace([np.inf, -np.inf], 0, inplace=True)

# df['Marks'] = df['Marks'].astype(str)

# create combo key
df_comp['Property Desc'] = df_comp['postcode']+' ('+df_comp['suburb']+') : '+df_comp['property_type']+' '+df_comp['bedrooms'].astype(str)+' bedrooms'

# del R_INFO_SUBSET_MED, R_INFO_SUBSET_FREQ, A_INFO_SUBSET_MED, A_INFO_SUBSET_FREQ
# 7303

In [40]:
# temp download file for testing
from datetime import datetime as dt
dte = dt.now().strftime('%Y%m%d')
fname = "/Users/joezhou/Downloads/REALESTATE_SUBURB_FOCUS{}.xlsx".format(dte)
df_comp.to_excel(fname)